# Mitiq and Qrack approximate Clifford+RZ simulation

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

You also need the `mitiq` package. `unitaryfund/mitiq` is a Python package for error mitigation. It relies intermediate representations of circuits like Qiskit's `QuantumCircuit` or Cirq's `Circuit`. In the example below, we use Qiskit's `QuantumCircuit`.

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os
import math
import numpy as np
import random

from pyqrack import QrackSimulator, Pauli
from mitiq import zne
from qiskit.circuit.quantumcircuit import QuantumCircuit
from mitiq.zne.scaling.folding import fold_global
from mitiq.zne.inference import RichardsonFactory

# Change these according to your OpenCL devices and system specifications:
os.environ['QRACK_MAX_PAGE_QB']='27'
os.environ['QRACK_MAX_ALLOC_MB']='15872'
os.environ['QRACK_MAX_PAGING_QB']='30'
os.environ['QRACK_MAX_CPU_QB']='-1'
# os.environ['QRACK_QUNITMULTI_DEVICES']='1'
# os.environ['QRACK_QPAGER_DEVICES']='1'

## Why mitigate Qrack simulation outputs?

Unitary Fund's "Qrack" quantum computer simulator framework simulates in the "ideal," by default, allowing users to simulate measuring "physically observable" expectation values with no systematic approximation techniques involved. However, we know that "ideal" quantum computer simulation can scale exponentially in classical computer resource usage. Sometimes, it's acceptable that our results are instead only _approximate_, so long as this allows us to reduce simulation resource footprint. To this end, Qrack offers several approximation techniques that sacrifice "fidelity" to recover a smaller peak simulation resource footprint, including "Schmidt decomposition rounding parameter" ("SDRP") and "near-Clifford" quantum circuit approximation techhniques. (We will focus on "near-Clifford" techniques that approximate large circuits with a "reduced density matrix," in this example.) Depending on available system resources, these approximation techniques might lead to a maximum achievable fidelity that is signficantly less than 100%, while employing Mitiq in addition, to mitigate the reduced-fidelity results, can trade off additional execution time, without significant additional memory footprint, to return higher fidelity results from approximate Qrack simulation!

## The Qrack executor

As with every Mitiq executor, we need a representation of our quantum circuit in a language or API that Mitiq knows how to interpret and is also compatible with our back-end execution target. Happily, Qrack provides "plugins" for both Qiskit and Cirq, circuits from either of which can be understood and used by both Mitiq and Qrack! (Click the links below to see the "plugin" repositories, and both are available for download via `pip`.)
- [Qiskit Qrack Provider](https://github.com/vm6502q/qiskit-qrack-provider)
- [Cirq Qrack Plugin](https://github.com/vm6502q/cirq-qrack)

**In fact, Qrack can "natively" run Qiskit circuits without reliance on any "provider" system at all!** This allows full, general use of the Qrack (and PyQrack) API, without any limitations imposed by the design of other frameworks for standardizing interchangeable back ends, while providing the full, familiar expressiveness and compatibility of alternative circuit definition APIs. Given the comparative difficulty of supporting an interface on non-default approximation techniques from Qrack in other frameworks designed for hardware (and different simulators), the easiest and most typical way to mitigate approximate Qrack simulation might be through Qrack's "native" parsing capabilities for Qiskit circuits.

Our first task is then to create an executor for Mitiq that accepts a Qiskit circuit and then simulates it and outputs an expectation value with Qrack. In this case, we will directly mitigate the fidelity of a "random circuit sampling" ("RCS") "mirror" circuit, which returns to exactly its original starting initialization by the end of the circuit "mirror" in the case of ideal simulation. (This way, for this example, we can exactly know the "ideal" result of our circuit without resort to costly and time-consuming ideal simulation.)

We could simply calculate and directly mitigate the fidelity (which can be exactly known in general from querying the end-result probability of a starting-state eigenstate). However, the developers of Qrack suggest in general that any "regression" or "extrapolation" on a _bounded_ expectation value interval be transformed to an _unbounded_ interval extrapolation through the use of "[logit()](https://en.wikipedia.org/wiki/Logit)" and "[expit()](https://en.wikipedia.org/wiki/Logistic_function)" functions. (This way, the assumptions of "least squares regression" can be self-consistently satisfied without "heteroscedasticity" due to "ceiling" and "floor" effects, which was a primary motiviation for the historical introduction of the "logit()" function in regression modeling, in the first place.)

These are just our "`logit()`" and "`expit()`" functions:

In [2]:
def logit(x):
    # Theoretically, these limit points are "infinite,"
    # but precision caps out between 36 and 37:
    if x > (1 - 6e-17):
        return 36
    # For the negative limit, the precisions caps out
    # between -37 and -38
    elif x < 1e-17:
        return -37
    return max(-37, min(36, np.log(x / (1 - x))))

def expit(x):
    # Theoretically, these limit points are "infinite,"
    # but precision caps out between 36 and 37:
    if x >= 37:
        return 1.0
    # For the negative limit, the precisions caps out
    # between -37 and -38
    elif x <= -38:
        return 0.0
    return 1 / (1 + np.exp(-x))

Putting it all together, a typical Qrack executor, for Mitiq, to calculate the (`logit()`) fidelity of a mirror circuit, can look like this:

In [3]:
def execute(circuit):
    """Returns the mirror circuit expectation value for unsigned integer overall bit string."""

    circ = circuit.compose(circuit.inverse())

    # This is a typical Qrack "layer stack" for near-Clifford techniques.
    qsim = QrackSimulator(circ.width(), isTensorNetwork=False, isSchmidtDecompose=False, isOpenCL=False)

    qsim.run_qiskit_circuit(circ, 0)

    qubits = list(range(width))
    perm = [False] * width

    # This is fidelity of the mirror circuit.
    # (We average "optimistic" and "pessimistic" cases of "rounding" optimizations.)
    p = (qsim.prob_perm_rdm(qubits, perm, r=True) + qsim.prob_perm_rdm(qubits, perm, r=False)) / 2

    # So as not to exceed floor at 0.0 and ceiling at 1.0, (assuming 0 < p < 1,)
    # we mitigate its logit function value (https://en.wikipedia.org/wiki/Logit)
    return logit(p)

## Applying to a mirror RCS case

Finally, if we follow an RCS protocol to generate a random circuit trial, we can form its overall mirror, then measure and mitigate the overall fidelity.

In [4]:
# Circuit parameters:

# Logical qubit count:
width = 36
# Circuit layer depth (x2 for mirror):
depth = 18
# Number of non-Clifford phase gates (x2 for mirror)
max_magic = 9

In [5]:
def cx(circ, q1, q2):
    circ.cx(q1, q2)

def cy(circ, q1, q2):
    circ.cy(q1, q2)

def cz(circ, q1, q2):
    circ.cz(q1, q2)

def acx(circ, q1, q2):
    circ.x(q1)
    circ.cx(q1, q2)
    circ.x(q1)

def acy(circ, q1, q2):
    circ.x(q1)
    circ.cy(q1, q2)

def acz(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

def nswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def pswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)   

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)
    circ.s(q1)
    circ.s(q2)

def iiswap(circ, q1, q2):
    circ.sdg(q2)
    circ.sdg(q1)
    circ.cz(q1, q2)
    circ.swap(q1, q2)

def random_circuit(width, depth, magic):
    circuit = QuantumCircuit(width)

    magic_fraction = 3 * width * depth / max_magic
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz

    for i in range(depth):
        # Single bit gates
        for j in range(width):
            for _ in range(3):
                # We're trying to cover 3 Pauli axes
                # with Euler angle axes x-z-x. 
                circuit.h(j)

                # We can trace out a quarter rotations around the Bloch sphere with stabilizer.
                rnd = random.randint(0, 3)
                if rnd & 1:
                    circuit.s(j)
                if rnd & 2:
                    circuit.z(j)

                # For each axis, there is a chance of "magic."
                if (magic > 0) and ((magic_fraction * random.random()) < 1):
                    angle = random.uniform(0, math.pi / 2)
                    circuit.rz(angle, j)
                    magic -= 1

        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                g(circuit, b1, b2)
    
    return circuit

Run this next cell several times to produce a noisy value, (optionally tuning the width, depth, and magic settings above,) and run the last cell to mitigate with Mitiq.

In [6]:
%%time

circ = random_circuit(width, depth, max_magic)
print("Raw gate count: ", sum(dict(circ.count_ops()).values()))

qsim = QrackSimulator(width, isTensorNetwork=False, isSchmidtDecompose=False, isOpenCL=False)
raw_fidelity = expit(execute(circ))

print("Raw fidelity: ", raw_fidelity)

Raw gate count:  4428
Raw fidelity:  0.03320312500000001
CPU times: user 970 ms, sys: 412 ms, total: 1.38 s
Wall time: 1.4 s


For Qrack near-Clifford approximation techniques, it is only non-Clifford gates that ultimately introduce error and reduce fidelity. However, this does not happen "locally" immediately upon application of the gate, but rather at the end of the circuit when an expectation value or measurements are requested as output. Hence, with zero-noise extrapolation, we opt to use the **global folding** technique of noise scaling, as local folding of non-Clifford gates will produce no overall effect on Qrack fidelity. (Overall, mitigation of Qrack can be "hit-or-miss," but then it can be a "hit.")

In [7]:
%%time

scale_count = 5
max_scale = 3
factory = RichardsonFactory(scale_factors=[(1 + (max_scale - 1) * x / scale_count) for x in range(0, scale_count)])
mitigated_fidelity = expit(zne.execute_with_zne(circ, execute, factory = factory, scale_noise = fold_global))

print("Raw fidelity: ", raw_fidelity)
print("Fidelity with ZNE: ", mitigated_fidelity)
print("Width/depth/magic (before circuit mirror): ", width, "/", depth, "/", max_magic)

Raw fidelity:  0.03320312500000001
Fidelity with ZNE:  0.03320312500000291
Width/depth/magic (before circuit mirror):  36 / 18 / 9
CPU times: user 25.7 s, sys: 12.1 s, total: 37.7 s
Wall time: 38 s
